# Heisig Mnemonic Generator Demo

Demonstrates character decomposition and explanation generation without Anki.

In [1]:
import json
from IPython.display import HTML, display

with open('heisig_addon/data/heisig_data.json', encoding='utf-8') as f:
    data = json.load(f)

print(f'Loaded {len(data)} characters')

Loaded 5397 characters


In [2]:
import sys, os
sys.path.insert(0, os.path.join(os.getcwd(), 'heisig_addon'))
from llm import generate_story, SYSTEM_PROMPT

CARD_CSS = """
<style>
.heisig-card {
  font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;
  background: #fafafa;
  border-radius: 12px;
  padding: 24px;
  max-width: 420px;
  margin: 12px 0;
  border: 1px solid #e0e0e0;
  text-align: center;
}
.heisig-card .char { font-size: 96px; line-height: 1.2; }
.heisig-card .keyword { font-size: 28px; font-weight: 600; color: #1a1a2e; margin: 8px 0; }
.heisig-card .nums { font-size: 13px; color: #888; }
.heisig-card .detail { font-size: 15px; color: #555; margin: 4px 0; text-align: left; }
.heisig-card .label { font-weight: 600; color: #333; }
.heisig-card .story { font-style: italic; color: #333; margin-top: 8px; text-align: left; line-height: 1.5; }
.heisig-card hr { border: none; border-top: 1px solid #e0e0e0; margin: 12px 0; }
</style>
"""
display(HTML(CARD_CSS))

def show(char, story=None):
    info = data.get(char)
    if not info:
        display(HTML(f'<p>Character <b>{char}</b> not found.</p>'))
        return
    
    nums = []
    for key, label in [('RSH_number', 'RSH'), ('RTH_number', 'RTH'), ('RTK_number', 'RTK')]:
        v = info.get(key, '')
        if v:
            nums.append(f'{label} #{v}')
    
    parts = [f'<div class="heisig-card"><div class="char">{char}</div>']
    parts.append(f'<div class="keyword">{info.get("keyword", "")}</div>')
    if nums:
        parts.append(f'<div class="nums">{", ".join(nums)}</div>')
    parts.append('<hr>')
    if info.get('reading'):
        parts.append(f'<div class="detail"><span class="label">Reading:</span> {info["reading"]}</div>')
    if info.get('components_detail'):
        parts.append(f'<div class="detail"><span class="label">Components:</span> {info["components_detail"]}</div>')
    if info.get('spatial'):
        parts.append(f'<div class="detail"><span class="label">Layout:</span> {info["spatial"]}</div>')
    if info.get('decomposition'):
        parts.append(f'<div class="detail"><span class="label">Decomposition:</span> {info["decomposition"]}</div>')
    if story:
        parts.append(f'<hr><div class="story">{story}</div>')
    parts.append('</div>')
    display(HTML(''.join(parts)))

def show_with_story(char, provider="gemini", api_key=None, model=None):
    """Show card with LLM-generated mnemonic story."""
    info = data.get(char)
    if not info:
        display(HTML(f'<p>Character <b>{char}</b> not found.</p>'))
        return
    if api_key is None:
        api_key = os.environ.get("GEMINI_API_KEY", "") or os.environ.get("ANTHROPIC_API_KEY", "") or os.environ.get("OPENAI_API_KEY", "")
        if not api_key:
            show(char, story="(Set GEMINI_API_KEY env var or pass api_key= to generate stories)")
            return
    if model is None:
        model = {"gemini": "gemini-2.0-flash", "anthropic": "claude-sonnet-4-20250514", "openai": "gpt-4o"}.get(provider, "gemini-2.0-flash")
    story = generate_story(char, info, provider, api_key, model)
    show(char, story=story)

In [3]:
show('学')

In [4]:
show('森')

In [5]:
# With LLM-generated mnemonic story (requires API key)
# Get a free Gemini key at https://aistudio.google.com/apikey
# Then: export GEMINI_API_KEY=your_key_here
show_with_story('学')

In [6]:
show_with_story('森')

In [7]:
show_with_story('愛')